# Example Notebook

This is an example notebook to demonstrate how your learning journal works.


## Code Examples

You can add code and it will be executed when the site is built:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create some example data
x = np.linspace(0, 10, 100)
y = np.sin(x)

plt.figure(figsize=(10, 4))
plt.plot(x, y)
plt.title('Example Plot')
plt.xlabel('x')
plt.ylabel('sin(x)')
plt.grid(True)
plt.show()


## Notes

Add your learning notes here. You can use:
- Markdown formatting
- Code cells
- Math equations: $E = mc^2$
- Images
- And much more!


# 4 Maximum Likelihood Estimation

Using the linear system we have recovered an initial guess of the states. From here, we wish to refine the estimate such that they are closer to their true and also recover the covariance of the initial state. This process will also take into account the noise properties of the system such that each measurement is weighted relative to the accuracy of its sensor. We can define the following state which we wish to optimize:

$$
\mathbf{x} = [\begin{matrix} \mathbf{x}_{I,1}^{\mathsf{T}} & \cdots & \mathbf{x}_{I,C}^{\mathsf{T}} & {}^{G}\mathbf{p}_{f,1}^{\mathsf{T}} & \cdots & {}^{G}\mathbf{p}_{f,N}^{\mathsf{T}} \end{matrix}]^{\mathsf{T}}
$$

$$
\mathbf{x}_{I,k} = [\begin{matrix} {}^{I_k}_{G}\bar{q}^{\mathsf{T}} & {}^{G}\mathbf{p}_{I_k}^{\mathsf{T}} & {}^{G}\mathbf{v}_{I_k}^{\mathsf{T}} & \mathbf{b}_{g,k}^{\mathsf{T}} & \mathbf{b}_{a,k}^{\mathsf{T}} \end{matrix}]^{\mathsf{T}}
$$


where we have $C$ inertial states at each imaging timestep, and $N$ environmental features. This is optimized using ceres-solver [20].

# 4.1 Inertial Measurement Model

Inertial readings are related to two bounding states through continuous preintegration [17] (the IMU frame $\{I_k\}$ has been shorten to $\{k\}$):

$$
{}^{G}\mathbf{p}_{k+1} = {}^{G}\mathbf{p}_{k} + {}^{G}\mathbf{v}_{k}\Delta T - \frac{1}{2}{}^{G}\mathbf{g}\Delta T^2 + {}^{k}_{G}\mathbf{R}^{\mathsf{T}}\mathbf{\alpha}_{k+1}
$$

$$
{}^{G}\mathbf{v}_{k+1} = {}^{G}\mathbf{v}_{k} - {}^{G}\mathbf{g}\Delta T + {}^{k}_{G}\mathbf{R}^{\mathsf{T}}\mathbf{\beta}_{k+1}
$$

$$
{}^{k+1}_{G}\bar{q} = {}^{k+1}_{k}\bar{q} \otimes {}^{k}_{G}\bar{q}
$$


where $\Delta T$ is the difference between the bounding pose timestamps ($t_k, t_{k+1}$) and ${}^{k}\mathbf{\alpha}_{k+1}, {}^{k}\mathbf{\beta}_{k+1}$ are defined by the following integrations of the IMU measurements:

$$
{}^{k}\mathbf{\alpha}_{k+1} = \int_{t_k}^{t_{k+1}} \int_{t_k}^{s} {}^{k}_{u}\mathbf{R} (\mathbf{a}_{m} - \mathbf{b}_{a} - \mathbf{n}_{a})\ duds
$$

$$
{}^{k}\mathbf{\beta}_{k+1} = \int_{t_k}^{t_{k+1}} {}^{k}_{u}\mathbf{R} (\mathbf{a}_{m} - \mathbf{b}_{a} - \mathbf{n}_{a})\ du
$$

We note that the preintegrated measurements, ${}^{k}\mathbf{\alpha}_{k+1}$, ${}^{k}\mathbf{\beta}_{k+1}$, ${}^{k+1}_{k}\bar{q}$ are dependent on the true biases. This dependency is addressed through a first order Taylor series expansion about the current bias estimates $\bar{\mathbf{b}}_w$ and $\bar{\mathbf{b}}_a$ at time $t_k$ (assumed to be known before linear initialization):

$$
^{k}\mathbf{\alpha}_{k+1} \approx {}^{k}\breve{\mathbf{\alpha}}_{k+1} + \frac{\partial \mathbf{\alpha}}{\partial \mathbf{b}_a}\Big|_{\bar{\mathbf{b}}_a}\Delta \mathbf{b}_a + \frac{\partial \mathbf{\alpha}}{\partial \mathbf{b}_w}\Big|_{\bar{\mathbf{b}}_w}\Delta \mathbf{b}_w
$$

$$
^{k}\mathbf{\beta}_{k+1} \approx {}^{k}\breve{\mathbf{\beta}}_{k+1} + \frac{\partial \mathbf{\beta}}{\partial \mathbf{b}_a}\Big|_{\bar{\mathbf{b}}_a}\Delta \mathbf{b}_a + \frac{\partial \mathbf{\beta}}{\partial \mathbf{b}_w}\Big|_{\bar{\mathbf{b}}_w}\Delta \mathbf{b}_w
$$

$$
^{k+1}_{k}\bar{q} \approx \bar{q}(\Delta \mathbf{b}_w)^{-1} \otimes {}^{k+1}_{k}\breve{\bar{q}}
$$


where ${}^{k}\breve{\mathbf{\alpha}}_{k+1}$, ${}^{k}\breve{\mathbf{\beta}}_{k+1}$, ${}^{k+1}_{k}\breve{\bar{q}}$ are the preintegrated measurements evaluated at the current bias estimates. In particular, ${}^{k+1}_{k}\breve{\bar{q}}$ can be found using the zeroth order quaternion integrator [21]. The quaternion which models multiplicative orientation corrections due to linearized bias change is:

$$
\bar{q}(\Delta \mathbf{b}_w) = \begin{bmatrix} \frac{\mathbf{\theta}}{\|\mathbf{\theta}\|} \sin \frac{\|\mathbf{\theta}\|}{2} \\ \cos \frac{\|\mathbf{\theta}\|}{2} \end{bmatrix}
$$

$$
\mathbf{\theta} = \frac{\partial \bar{q}}{\partial \mathbf{b}_w}\Big|_{\bar{\mathbf{b}}_w}\ (\mathbf{b}_{w}(k) - \bar{\mathbf{b}}_w)
$$


where $\Delta \mathbf{b}_w := \mathbf{b}_w(k) - \bar{\mathbf{b}}_w$ and $\Delta \mathbf{b}_a := \mathbf{b}_a(k) - \bar{\mathbf{b}}_a$ are the differences between the true biases and the current bias estimate used as the linearization point. The new preintegration measurements can now be computed once and changes in the bias estimates can be taken into account through the above Taylor series. The final measurement residual is as follows:

$$
r_I(\mathbf{x}) = \begin{bmatrix}
2 \text{vec} \left( {}^{k+1}_{V}\bar{q} \otimes \bar{q}_{V}^{-1} \otimes {}^{k+1}_{k}\tilde{\bar{q}}^{-1} \otimes \bar{q}(\Delta \mathbf{b}_w) \right) \\
\mathbf{b}_{w,k+1} - \mathbf{b}_{w,k} \\
\left( {}^{k}_{V}\mathbf{R} \left( \mathbf{v}_{k+1} - \mathbf{v}_{k} + {}^{V}_{G}\mathbf{R}^{\mathsf{T}}\mathbf{g}\Delta T \right) \right) \\
\left( -{}^{k}\tilde{\mathbf{\beta}}_{k+1} - \frac{\partial \mathbf{\beta}}{\partial \mathbf{b}_a}\Big|_{\bar{\mathbf{b}}_a}\Delta \mathbf{b}_a - \frac{\partial \mathbf{\beta}}{\partial \mathbf{b}_w}\Big|_{\bar{\mathbf{b}}_w}\Delta \mathbf{b}_w \right) \\
\mathbf{b}_{a,k+1} - \mathbf{b}_{a,k} \\
\left( {}^{k}_{V}\mathbf{R} \left( \mathbf{p}_{k+1} - \mathbf{p}_{k} - \mathbf{v}_{k}\Delta T + \frac{1}{2}{}^{V}_{G}\mathbf{R}^{\mathsf{T}}\mathbf{g}\Delta T^2 \right) \right) \\
\left( -{}^{k}\tilde{\mathbf{\alpha}}_{k+1} - \frac{\partial \mathbf{\alpha}}{\partial \mathbf{b}_a}\Big|_{\bar{\mathbf{b}}_a}\Delta \mathbf{b}_a - \frac{\partial \mathbf{\alpha}}{\partial \mathbf{b}_w}\Big|_{\bar{\mathbf{b}}_w}\Delta \mathbf{b}_w \right)
\end{bmatrix}
$$

where vec($\cdot$) returns the vector portion of the quaternion (i.e., the top three elements) and the bias errors are the difference between biases in the bounding states.

We use combined continuous preintegration factors that included both the inertial and bias errors together and relate to the full 15 degree-of-freedom state (see Equation 53). This combined continuous preintegration factor better models the measurement error state dynamics due to bias drift over the integration interval. Thus we have the following:

$$
C_I \triangleq \|0 - r_I(\mathbf{x})\|^2_{\mathbf{P}^{-1}} \tag{64}
$$

where $\mathbf{P}$ is the measurement covariance and we refer the reader to the original paper for it and the state Jacobians [17, 22].


## Covariance
In/At:

$$
\dot{r} = Fr + Gn
$$

$$
P_{K_1} = F_{K_1} P_{\text{meas}} + P_{\text{meas}} F_{K_1}^{\mathsf{T}} + G_{K_1} Q_C G_{K_1}^{\mathsf{T}}
$$
$\to$ Covariance at start pt.

$$
P_{K_2} = P_{\text{meas}} + \dot{P}_{K_1} \frac{\Delta t}{2}
$$
$\to$ Covariance at mid pt

$$
\dot{P}_{K_2} = F_{K_2} P_{K_2} + P_{K_2} F_{K_2}^{\mathsf{T}} + G_{K_2} Q_C G_{K_2}^{\mathsf{T}}
$$

$$
F_{K_3} = F_{K_2} \quad G_{K_3} = G_{K_2}
$$

$$
P_{K_3} = P_{\text{meas}} + \dot{P}_{K_2} \frac{\Delta t}{2}
$$

$$
\dot{P}_{K_3} = F_{K_3} P_{K_3} + P_{K_3} F_{K_3}^{\mathsf{T}} + G_{K_3} Q_C G_{K_3}^{\mathsf{T}}
$$

$$
P_{K_4} = P_{\text{meas}} + \dot{P}_{K_3} \Delta t
$$
$\to$ Covariance at end pt

$$
\dot{P}_{K_4} = F_{K_4} P_{K_4} + P_{K_4} F_{K_4}^{\mathsf{T}} + G_{K_4} Q_C G_{K_4}^{\mathsf{T}}
$$

$$
P_{\text{meas}} = P_{\text{meas}} + \frac{\Delta t}{6} (\dot{P}_{K_1} + 2\dot{P}_{K_2} + 2\dot{P}_{K_3} + \dot{P}_{K_4})
$$

$$
X_{n+1} = X_n + \frac{\Delta t}{6} (K_1 + 2K_2 + 2K_3 + K_4)
$$

Full derivations for jacobians here:
https://pgeneva.com/downloads/reports/tr_init.pdf
https://udel.edu/~ghuang/papers/tr_cpi.pdf

# 4.2 Camera Measurement Model

We follow the camera model used in OpenVINS [1]. Each observation of a feature can be written as a function of the state by:

$$
\mathbf{z}_{c,k} = h_c(\mathbf{x}) + \mathbf{n}_{c,k} \tag{65}
$$

$$
= h_d(\mathbf{z}_{n,k}, \zeta) + \mathbf{n}_{c,k} \tag{66}
$$

$$
= h_d(h_p({}^{C_k}\mathbf{p}_f), \zeta) + \mathbf{n}_{c,k} \tag{67}
$$

$$
= h_d(h_p(h_t({}^{G}\mathbf{p}_f, {}^{C_k}_{G}\mathbf{R}, {}^{G}\mathbf{p}_{C_k})), \zeta) + \mathbf{n}_{c,k} \tag{68}
$$

where $\mathbf{z}_{c,k}$ is the raw uv pixel coordinate; $\mathbf{n}_{c,k}$ the raw pixel noise and typically assumed to be zero-mean white Gaussian $\mathbf{R}_{c,i}$; $\mathbf{z}_{n,k}$ is the normalized undistorted uv measurement; ${}^{C_k}\mathbf{p}_f$ is the landmark position in the current camera frame; ${}^{G}\mathbf{p}_f$ is the landmark position in the global frame and depending on its representation may also be a function of state elements; and $\{{}^{C_k}_{G}\mathbf{R}, {}^{G}\mathbf{p}_{C_k}\}$ denotes the current camera pose (position and orientation) in the global frame which are related to $\{{}^{I_k}_{G}\mathbf{R}, {}^{G}\mathbf{p}_{I_k}\}$ through the extrinsic calibration.

The measurement functions $h_d, h_p$, and $h_t$ correspond to the intrinsic distortion, projection, and transformation functions and the corresponding measurement Jacobians can be computed through a simple chain rule. We can then define the following cost for feature observations:

$$
C_C \triangleq \|\mathbf{z}_{c} - h_c(\mathbf{x})\|^2_{\mathbf{R}_{c,i}^{-1}} \tag{69}
$$


Measurement Function Overview

$$ \mathbf{z}_{k}=h_{d}(\mathbf{z}_{n,k},\zeta )$$
The distortion function that takes normalized coordinates and maps it into distorted uv coordinates
$$\mathbf{z}_{n,k}=h_{p}({}^{C_{k}}\mathbf{p}_{f})$$
The projection function that takes a 3D point in the image and converts it into the normalized uv coordinates
$${}^{C_{k}}\mathbf{p}_{f}=h_{t}({}^{G}\mathbf{p}_{f},{}_{G}^{C_{k}}\mathbf{R},{}^{G}\mathbf{p}_{C_{k}})$$
Transforming a feature's position in the global frame into the current camera frame
$${}^{G}\mathbf{p}_{f}=h_{r}(\mathbf{\lambda },\dots )$$
Converting from a feature representation to a 3D feature in the global frame